In [ ]:
# Install necessary packages, correct one
!pip install -q transformers pillow gradio torch


In [ ]:

# Import libraries
import os
import gradio as gr
# Create the "images" folder and add some placeholder images if needed
os.makedirs("images", exist_ok=True)

from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM

import torch
#torch.cuda.empty_cache()

# Load caption model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cpu")

#Load story model
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
story_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2").to("cuda")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def generate_caption(image):
    inputs = processor(images=image, return_tensors="pt").to("cpu")
    output = caption_model.generate(**inputs, max_length=50)
    caption = processor.decode(output[0], skip_special_tokens=True).capitalize()
    if not caption.endswith("."):
        caption += "."
    return caption

In [ ]:
def generate_story(image, style):
    caption = generate_caption(image)
    prompt = f"Write a story in the style of {style} based on: {caption}"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

    outputs = story_model.generate(
        input_ids=input_ids,
        min_length=100,
        max_length=200,
        num_beams=5,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        no_repeat_ngram_size=2
    )
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    story = full_output[len(prompt):].strip()
    if not story.endswith("."):
       story += "."

    return caption, story

In [ ]:
with gr.Blocks(css="""
/* 🌙 Dark Mode Background */
.gradio-container {
    background-color: #18181B;
    color: #FFFFFF;
    font-family: 'Poppins', sans-serif;
    padding: 20px;
    display: flex;
    justify-content: center;
}

/* Stack content vertically and center */
#main-column {
    display: flex;
    flex-direction: column;
    align-items: center;
    max-width: 800px;
    width: 100%;
    margin: auto;
}

/* 🖼️ Image Styling */
.gr-image {
    border-radius: 12px;
    box-shadow: 4px 4px 10px rgba(255, 255, 255, 0.2);
}

/* ✏️ Textbox Enhancements */
.gr-textbox {
    width: 90%;
    font-size: 18px;
    padding: 10px;
    border: 2px solid #4A4A4D;
}

/* 🎨 Button Customization */
.gr-button {
    background-color: #5A67D8;
    color: pink;
    font-size: 16px;
    padding: 12px 18px;
    border-radius: 8px;
    transition: 0.2s ease-in-out;
}

/* ✨ Refine Labels */
label {
    font-weight: bold;
    color: #D1D5DB;
}
""") as demo:

    with gr.Column(elem_id="main-column"):
        gr.Markdown("<h2 style='color: #EAB308;'>✏️ Create your story</h2><p style='color: #9CA3AF;'>Upload your image and get a caption + story.</p>")

        image_input = gr.Image(type="pil", label="Upload an image")
        style_input = gr.Radio(["Poetic", "Sci-Fi", "Child-friendly"], label="Choose a style")
        generate_btn = gr.Button("Generate")

        caption_output = gr.Textbox(label="Generated Caption")
        story_output = gr.Textbox(label="Generated Story")

        generate_btn.click(fn=generate_story, inputs=[image_input, style_input],
                           outputs=[caption_output, story_output])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://56d5178e3db4db4cba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
